In [38]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

from sklearn.preprocessing import StandardScaler

In [39]:
data = pd.read_excel("/content/15min_dataset_pre_NN.xlsx", sheet_name="Sheet1")
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values


In [40]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


In [41]:
scaler = StandardScaler()
#X = (X - X.mean()) / X.std()
X_scaled = scaler.fit_transform(X)

#scaler_y = MinMaxScaler()
#y_scaled = scaler_y.fit_transform(y_original)

# Splitting the dataset into training, testing, and validation sets
X_train, X_temp, y_train, y_temp = train_test_split(X_scaled, y, test_size=0.3, random_state=1234)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=1234)

y_train = np.array(y_train)
y_val = np.array(y_val)
y_test = np.array(y_test)

X_train = torch.tensor(X_train, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

train = TensorDataset(X_train, y_train)
test = TensorDataset(X_test, y_test)
val = TensorDataset(X_val, y_val)

# Vytvorenie dataloaders pre trénovaciu a testovaciu sadu
train_loader = DataLoader(train, batch_size=64, shuffle=True)
test_loader = DataLoader(test, batch_size=64, shuffle=False)
validation_loader = DataLoader(val, batch_size=64, shuffle=False)

In [42]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv1d(1, 32, kernel_size=3)  # Change to 1D convolution
        self.conv2 = nn.Conv1d(32, 64, kernel_size=3)
        self.fc1 = nn.Linear(64 * 22, 128)  # Adjusted input size based on kernel size
        self.fc2 = nn.Linear(128, 10)  # Adjust output size according to your task

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = torch.flatten(x, 1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = CNN()


In [43]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [46]:
# Define the number of epochs
epochs = 500

# Define lists to store training and validation losses
train_losses = []
val_losses = []

# Training loop
for epoch in range(epochs):
    # Set model to train mode
    model.train()
    train_loss = 0.0

    # Iterate over the training data
    for inputs, labels in train_loader:
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(data)  # Add an extra dimension for the channel
        loss = criterion(outputs, labels.long())

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Accumulate the training loss
        train_loss += loss.item() * inputs.size(0)

    # Calculate average training loss for the epoch
    train_loss /= len(train_loader.dataset)
    train_losses.append(train_loss)

    # Set model to evaluation mode
    model.eval()
    val_loss = 0.0

    # Turn off gradients for validation
    with torch.no_grad():
        # Iterate over the validation data
        for inputs, labels in validation_loader:
            # Forward pass
            outputs = model(inputs.unsqueeze(1))  # Add an extra dimension for the channel
            loss = criterion(outputs, labels.long())

            # Accumulate the validation loss
            val_loss += loss.item() * inputs.size(0)

    # Calculate average validation loss for the epoch
    val_loss /= len(validation_loader.dataset)
    val_losses.append(val_loss)

    # Print progress
    print(f'Epoch [{epoch + 1}/{epochs}], Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}')

# After training, you can evaluate the model on the test set if needed
test_loss = 0.0
correct = 0
total = 0

# Turn off gradients for testing
with torch.no_grad():
    # Iterate over the test data
    for inputs, labels in test_loader:
        # Forward pass
        outputs = model(inputs.unsqueeze(1))  # Add an extra dimension for the channel
        loss = criterion(outputs, labels.long())

        # Accumulate the test loss
        test_loss += loss.item() * inputs.size(0)

        # Get the predicted labels
        _, predicted = torch.max(outputs, 1)
        # Total number of labels
        total += labels.size(0)
        # Total correct predictions
        correct += (predicted == labels).sum().item()

# Calculate average test loss
test_loss /= len(test_loader.dataset)

# Calculate accuracy
accuracy = correct / total
print(f'Test Loss: {test_loss:.4f}, Accuracy: {accuracy:.4f}')


TypeError: conv1d() received an invalid combination of arguments - got (DataFrame, Parameter, Parameter, tuple, tuple, tuple, int), but expected one of:
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, tuple of ints padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!DataFrame!, !Parameter!, !Parameter!, !tuple of (int,)!, !tuple of (int,)!, !tuple of (int,)!, int)
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, str padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!DataFrame!, !Parameter!, !Parameter!, !tuple of (int,)!, !tuple of (int,)!, !tuple of (int,)!, int)
